In [ ]:
import cv2
import numpy as np
import tkinter as tk
from PIL import Image, ImageTk
import matplotlib.pyplot as plt
from collections import deque

In [ ]:
def detect_bills(image, realtime=False):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    annotated = image.copy()
    total = 0
    counts = {20:0, 50:0, 100:0, 200:0, 500:0, 1000:0}

    bill_colors = {
        20:   ([5, 100, 100],   [20, 255, 255]),   # orange-brown
        50:   ([160, 100, 100], [179, 255, 255]),  # red
        100:  ([125, 80, 80],   [150, 255, 255]),  # violet
        200:  ([40, 80, 80],    [80, 255, 255]),   # green
        500:  ([20, 80, 80],    [35, 255, 255]),   # yellow
        1000: ([90, 80, 80],    [120, 255, 255])   # blue
    }

    for val, (lower, upper) in bill_colors.items():
        lower = np.array(lower, dtype=np.uint8)
        upper = np.array(upper, dtype=np.uint8)

        mask = cv2.inRange(hsv, lower, upper)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5,5), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((7,7), np.uint8))

        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area < 30000 or area > 300000:  
                continue

            rect = cv2.minAreaRect(cnt)
            (x, y), (w, h), angle = rect
            aspect_ratio = max(w, h) / (min(w, h) + 1e-5)

            if aspect_ratio < 2.0 or aspect_ratio > 3.5:
                continue

            box = cv2.boxPoints(rect)
            box = np.round(box).astype(int)
            mask_roi = np.zeros(mask.shape, dtype=np.uint8)
            cv2.drawContours(mask_roi, [box], -1, 255, -1)
            bill_area = cv2.countNonZero(mask_roi)
            color_area = cv2.countNonZero(cv2.bitwise_and(mask, mask, mask=mask_roi))

            if bill_area == 0:
                continue

            color_ratio = color_area / bill_area
            if color_ratio < 0.3: 
                continue

            counts[val] += 1
            total += val

            cv2.drawContours(annotated, [box], 0, (0, 255, 0), 3)
            cv2.putText(annotated, f"₱{val}", (int(x)-40, int(y)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.putText(annotated, f"Total: ₱{total}", (20, 40), 
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)

    return annotated, counts, total


In [ ]:
class BillApp:
    def __init__(self, root, camera_index=0):
        self.root = root
        self.root.title("Philippine Peso Bill Detector")

        self.video_label = tk.Label(root)
        self.video_label.pack()

        controls = tk.Frame(root)
        controls.pack(pady=6)

        self.start_button = tk.Button(controls, text="Start Camera", command=self.start_camera)
        self.start_button.grid(row=0, column=0, padx=4)

        self.stop_button = tk.Button(controls, text="Stop Camera", command=self.stop_camera)
        self.stop_button.grid(row=0, column=1, padx=4)

        self.snapshot_button = tk.Button(controls, text="Save Snapshot", command=self.save_snapshot)
        self.snapshot_button.grid(row=0, column=2, padx=4)

        self.camera_index = camera_index

        self.cap = None
        self.frame = None
        self.running = False

    def start_camera(self):
        if not self.running:
            self.cap = cv2.VideoCapture(self.camera_index)
            self.cap.set(cv2.CAP_PROP_FRAME_WIDTH,  1280)
            self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
            self.running = True
            self.update_frame()

    def stop_camera(self):
        if self.running:
            self.running = False
            if self.cap is not None:
                self.cap.release()

    def save_snapshot(self):
        if self.frame is not None:
            cv2.imwrite("bills_snapshot.png", cv2.cvtColor(self.frame, cv2.COLOR_BGR2RGB))
            print("Snapshot saved as bills_snapshot.png")

    def update_frame(self):
        if self.running and self.cap is not None:
            ret, frame = self.cap.read()
            if ret:
                annotated, counts, total = detect_bills(frame, realtime=True)
                self.frame = annotated
                img = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(img)
                imgtk = ImageTk.PhotoImage(image=img)
                self.video_label.imgtk = imgtk
                self.video_label.configure(image=imgtk)
        self.root.after(33, self.update_frame)

In [ ]:
root = tk.Tk()
app = BillApp(root, camera_index=1)
root.mainloop()